In [7]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
import xml.etree.ElementTree as xet
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [9]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [11]:
filename = df['filepath'][0]

In [13]:
def getFileName(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images', filename_image)
    return filepath_image


In [15]:
getFileName(filename)

'./images\\N1.jpeg'

In [17]:
image_path = list(df['filepath'].apply(getFileName))
image_path

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

### Verify image and output

In [20]:
file_path = image_path[0]
file_path

'./images\\N1.jpeg'

In [22]:
img = cv2.imread(file_path)

cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
# 1093	1396	645	727
cv2.rectangle(img, (1093, 645), (1396, 727), (0, 255, 0), 3)
cv2.namedWindow('example', cv2.WINDOW_NORMAL)
cv2.imshow('example', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Processing

In [25]:
labels = df.iloc[:,1:].values

In [26]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    #preprocessig
    load_image = load_img(image, target_size = (224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 
    #normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = nxmin, nxmax, nymin, nymax
    #append it to list
    data.append(norm_load_image_arr)
    output.append(label_norm)

### Data splitting

In [28]:
x = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [29]:
x.shape, y.shape


((225, 224, 224, 3), (225, 4))

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size= 0.8, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Deep Learning model

In [91]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [93]:
inception_resnet = InceptionResNetV2(weights = "imagenet", include_top = False, input_tensor = Input(shape=(224, 224, 3)))

inception_resnet.trainable = False
#______________________________________________________#
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation = "relu")(headmodel)
headmodel = Dense(250, activation = "relu")(headmodel)
headmodel = Dense(4, activation = "sigmoid")(headmodel)
#______________________________________________________#
model = Model(inputs=inception_resnet.input, outputs=headmodel)

In [94]:
# compile model
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_203 (Conv2D) │ (None, 111, 111,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_203[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_203      │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_204 (Conv2D) │ (None, 109, 109,  │      9,216 │ activation_203[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_204[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_204      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_205 (Conv2D) │ (None, 109, 109,  │     18,432 │ activation_204[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_205[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_205      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 54, 54,    │          0 │ activation_205[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_206 (Conv2D) │ (None, 54, 54,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_206[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_206      │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_207 (Conv2D) │ (None, 52, 52,    │    138,240 │ activation_206[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_207[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_207      │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

### model train

In [102]:
from tensorflow.keras.callbacks import TensorBoard

In [104]:
tfb = TensorBoard('object_detection')

In [106]:
import tensorflow as tf

x_train = tf.constant(x_train)
x_test = tf.constant(x_test)

In [108]:
print(x_train.shape)  # Should match (batch_size, 224, 224, 3)
print(model.input_shape)  # Should match x_train shape

(180, 224, 224, 3)
(None, 224, 224, 3)


In [59]:
history = model.fit(x=x_train, y=y_train,batch_size=10, epochs=100, validation_data=(x_test, y_test),callbacks=[tfb])

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.1035 

D:\Anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


18/18 ━━━━━━━━━━━━━━━━━━━━ 91s 5s/step - loss: 0.1034 - val_loss: 0.0949
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step - loss: 0.0915 - val_loss: 0.0887
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - loss: 0.0857 - val_loss: 0.0982
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - loss: 0.0858 - val_loss: 0.0860
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - loss: 0.0803 - val_loss: 0.0882
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - loss: 0.0821 - val_loss: 0.0959
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - loss: 0.0847 - val_loss: 0.0933
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - loss: 0.0878 - val_loss: 0.0898
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - loss: 0.0775 - val_loss: 0.0872
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - loss: 0.0797 - val_loss: 0.0853
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - loss: 0.0776 - val_loss: 0.0854
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - loss: 0.0809 - val_lo

In [122]:
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
import numpy as np
x_train = preprocess_input(x_train)  # Assuming x_train is a numpy array
x_test = preprocess_input(x_test)  # Assuming x_test is a numpy array

In [124]:
history = model.fit(x=x_train, y=y_train, batch_size=5, epochs=20, validation_data=(x_test, y_test), callbacks=[tfb])

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - loss: 0.0236 - val_loss: 0.0215
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - loss: 0.0216 - val_loss: 0.0211
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - loss: 0.0216 - val_loss: 0.0218
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 0.0190 - val_loss: 0.0238
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - loss: 0.0217 - val_loss: 0.0216
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - loss: 0.0203 - val_loss: 0.0245
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - loss: 0.0222 - val_loss: 0.0232
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 54s 1s/step - loss: 0.0224 - val_loss: 0.0195
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - loss: 0.0189 - val_loss: 0.0198
Epoch 10/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - loss: 0.0208 - val_loss: 0.0209
Epoch 11/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - loss: 0.0222 - val_loss: 0.0219
Epoch 12/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - loss: 0.0224 - val_l

In [130]:
# model.save('./models/object_detection.h5')
import keras.saving
keras.saving.save_model(model, "./models/object_detection.keras")

In [128]:
# Save the model in .h5 format
model.save('./models/object_detection.h5')
